In [3]:
%%time

from datetime import datetime, timedelta, timezone
import json
import os
import re
import boto3
import time
from time import sleep
from threading import Thread

import pandas as pd
import json

from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer
from sagemaker.predictor import Predictor

from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig

CPU times: user 783 ms, sys: 144 ms, total: 927 ms
Wall time: 1.29 s


In [4]:
session = Session()
role = get_execution_role()
print("RoleArn:", role)
region = session.boto_region_name
print("Region:", region)

RoleArn: arn:aws:iam::623127157773:role/sagemaker-role
Region: ap-southeast-1


In [5]:
bucket='sagemaker-ap-southeast-1-623127157773'
prefix = 'sagemaker/iris-monitor'
data_capture_prefix = '{}/datacapture'.format(prefix)
s3_capture_upload_path = 's3://{}/{}'.format(bucket, data_capture_prefix)
reports_prefix = '{}/reports'.format(prefix)
s3_report_path = 's3://{}/{}'.format(bucket,reports_prefix)
model_monitor_prefix = '{}/modelmonitor'.format(prefix)+f"-iris-{datetime.utcnow():%Y-%m-%d-%H%M}"
model_monitor_results_path = 's3://{}/{}'.format(bucket,model_monitor_prefix)
code_prefix = '{}/code'.format(prefix)
print("Capture path: {}".format(s3_capture_upload_path))
print("Report path: {}".format(s3_report_path))

Capture path: s3://sagemaker-ap-southeast-1-623127157773/sagemaker/XGBoost-Recommendations-Monitor3/datacapture
Report path: s3://sagemaker-ap-southeast-1-623127157773/sagemaker/XGBoost-Recommendations-Monitor3/reports


In [6]:
model_url = 's3://sagemaker-ap-southeast-1-623127157773/output/sagemaker-decision-trees-2023-07-14-16-41-45-933/output/model.tar.gz'
model_name = f"iris-pred-model-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"
model = Model(image_uri='623127157773.dkr.ecr.ap-southeast-1.amazonaws.com/sagemaker-decision-trees:latest', model_data=model_url, role=role, sagemaker_session=session)

In [7]:
endpoint_name = f"iris-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"
print("EndpointName =", endpoint_name)
print(s3_capture_upload_path)
data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

EndpointName = iris-hashirama-model-quality-monitor-2023-07-13-0945
s3://sagemaker-ap-southeast-1-623127157773/sagemaker/XGBoost-Recommendations-Monitor3/datacapture
---!

In [8]:
predictor = Predictor(
    endpoint_name=predictor.endpoint, sagemaker_session=session, serializer=CSVSerializer()
)
# do prediction.
for i in range(0,10):
    print(predictor.predict([1,2,3,4]).decode("utf-8"))

In [9]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    volume_size_in_gb=3,
    max_runtime_in_seconds=3600,
)

my_monitor.suggest_baseline(
    baseline_dataset='s3://iris-data-for-sagemaker/iris-ori.csv',
    dataset_format=DatasetFormat.csv(header=True)
)

INFO:sagemaker:Creating processing-job with name baseline-suggestion-job-2023-07-13-09-48-12-716


.................................2023-07-13 09:53:42,557 - matplotlib.font_manager - INFO - Generating new fontManager, this may take some time...
2023-07-13 09:53:43.215252: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-13 09:53:43.215292: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-13 09:53:45.053282: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-13 09:53:45.053324: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-13 09:53:45.053358: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to 

In [10]:
from sagemaker.model_monitor import CronExpressionGenerator
from sagemaker.model_monitor import EndpointInput
from time import gmtime, strftime

response = my_monitor.create_monitoring_schedule(
    monitor_schedule_name='custom-iris-monitor',
    endpoint_input=predictor.endpoint,
    output_s3_uri=s3_report_path,
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: kustom-iris-monitor


In [15]:
# do prediction.
for i in range(0,3000):
    predictor.predict([5.8,6.3,7.2,8.1]).decode("utf-8")

In [12]:
desc_schedule_result = my_monitor.describe_schedule()
print("Schedule status: {}".format(desc_schedule_result["MonitoringScheduleStatus"]))

Schedule status: Scheduled


In [16]:
mon_executions = my_monitor.list_executions()
print(
    "We created a hourly schedule above that begins executions ON the hour (plus 0-20 min buffer.\nWe will have to wait till we hit the hour..."
)

while len(mon_executions) == 0:
    print("Waiting for the first execution to happen...")
    time.sleep(60)
    mon_executions = my_monitor.list_executions()

No executions found for schedule. monitoring_schedule_name: kustom-iris-monitor
We created a hourly schedule above that begins executions ON the hour (plus 0-20 min buffer.
We will have to wait till we hit the hour...
Waiting for the first execution to happen...
No executions found for schedule. monitoring_schedule_name: kustom-iris-monitor
Waiting for the first execution to happen...
No executions found for schedule. monitoring_schedule_name: kustom-iris-monitor
Waiting for the first execution to happen...
No executions found for schedule. monitoring_schedule_name: kustom-iris-monitor
Waiting for the first execution to happen...


In [17]:
latest_execution = mon_executions[-1]  # Latest execution's index is -1, second to last is -2, etc
time.sleep(60)
latest_execution.wait(logs=False)

print("Latest execution status: {}".format(latest_execution.describe()["ProcessingJobStatus"]))
print("Latest execution result: {}".format(latest_execution.describe()["ExitMessage"]))

latest_job = latest_execution.describe()
if latest_job["ProcessingJobStatus"] != "Completed":
    print(
        "====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures."
    )

!Latest execution status: Completed
Latest execution result: CompletedWithViolations: Job completed successfully with 8 violations.


In [18]:
report_uri = latest_execution.output.destination
print("Report Uri: {}".format(report_uri))

Report Uri: s3://sagemaker-ap-southeast-1-623127157773/model-monitor/baselining/baseline-suggestion-job-2023-07-09-13-58-47-805/results/iris-hashirama-model-quality-monitor-2023-07-13-0945/kustom-iris-monitor/2023/07/13/10


In [19]:
from urllib.parse import urlparse

s3uri = urlparse(report_uri)
report_bucket = s3uri.netloc
report_key = s3uri.path.lstrip("/")
print("Report bucket: {}".format(report_bucket))
print("Report key: {}".format(report_key))

s3_client = boto3.Session().client("s3")
result = s3_client.list_objects(Bucket=report_bucket, Prefix=report_key)
report_files = [report_file.get("Key") for report_file in result.get("Contents")]
print("Found Report Files:")
print("\n ".join(report_files))

Report bucket: sagemaker-ap-southeast-1-623127157773
Report key: model-monitor/baselining/baseline-suggestion-job-2023-07-09-13-58-47-805/results/iris-hashirama-model-quality-monitor-2023-07-13-0945/kustom-iris-monitor/2023/07/13/10
Found Report Files:
model-monitor/baselining/baseline-suggestion-job-2023-07-09-13-58-47-805/results/iris-hashirama-model-quality-monitor-2023-07-13-0945/kustom-iris-monitor/2023/07/13/10/constraint_violations.json
 model-monitor/baselining/baseline-suggestion-job-2023-07-09-13-58-47-805/results/iris-hashirama-model-quality-monitor-2023-07-13-0945/kustom-iris-monitor/2023/07/13/10/constraints.json
 model-monitor/baselining/baseline-suggestion-job-2023-07-09-13-58-47-805/results/iris-hashirama-model-quality-monitor-2023-07-13-0945/kustom-iris-monitor/2023/07/13/10/statistics.json


In [23]:
violations = my_monitor.latest_monitoring_constraint_violations()
pd.set_option("display.max_colwidth", None)
constraints_df = pd.json_normalize(violations.body_dict["violations"])
constraints_df.head(10)

,feature_name,constraint_check_type,description
0,variety,categorical_values_check,Categorical value match requirement is not met. Expected match: 100.0%. Observed: 0.0% of the values match the known values.
1,sepal_width,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 30.0% of data is Fractional."
2,petal_width,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 30.0% of data is Fractional."
3,sepal_length,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 0.0% of data is Fractional."
4,petal_length,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 30.0% of data is Fractional."
5,variety,data_type_check,"Data type match requirement is not met. Expected data type: String, Expected match: 100.0%. Observed: Only 0.0% of data is String."
6,sepal_width,baseline_drift_check,Baseline drift distance: 0.11818656128131999 exceeds threshold: 0.1
7,petal_width,baseline_drift_check,Baseline drift distance: 0.31350693574317356 exceeds threshold: 0.1


In [24]:
my_monitor.stop_monitoring_schedule()
my_monitor.delete_monitoring_schedule()


Stopping Monitoring Schedule with name: kustom-iris-monitor


In [25]:
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: iris-hashirama-model-quality-monitor-2023-07-13-0945
INFO:sagemaker:Deleting endpoint with name: iris-hashirama-model-quality-monitor-2023-07-13-0945
